<a href="https://colab.research.google.com/github/jbishop45/CS-7650/blob/project-1/TextClassification_release_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to https://aritter.github.io/CS-7650/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)

#Project \#1: Text Classification

In this assignment, you will implement the perceptron algorithm, and a simple, but competitive neural bag-of-words model, as described in [this paper](https://www.aclweb.org/anthology/P15-1162.pdf) for text classification.  You will train your models on a (provided) dataset of positive and negative movie reviews and report accuracy on a test set.

In this notebook, we provide you with starter code to read in the data and evaluate the performance of your models.  After completing the instructions below, please follow the instructions at the end to submit your notebook and other files to Gradescope.

Make sure to make a copy of this notebook, so your changes are saved.

#Download the dataset

First you will need to download the IMDB dataset - to do this, simply run the cell below.  We have prepared a small version of the ACL IMDB dataset for you to use to help make your experiments faster.  The full dataset is available [here](https://ai.stanford.edu/~amaas/data/sentiment/), in case you are interested, but there is no need to use this for the assignment.

Note that files downloaded in Colab are only saved temporariliy - if your session reconnects you will need to re-download the files.  In case you need persistent storage, you can mount your Google drive folder like so:

```
from google.colab import drive
drive.mount('/content/drive')
```

You can also open a command line prompt by clicking on the shell icon on the left hand side of the page, and upload/download files from your local machine by clicking on the file icon.

In [ ]:
#Download the data

!wget -O aclImdb_small.tgz https://github.com/aritter/CS-7650-sp22/blob/master/slides/aclImdb_small.tgz?raw=true
!tar xvzf aclImdb_small.tgz > /dev/null

--2022-01-10 23:36:38--  https://github.com/aritter/CS-7650-sp22/blob/master/slides/aclImdb_small.tgz?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/aritter/CS-7650-sp22/raw/master/slides/aclImdb_small.tgz [following]
--2022-01-10 23:36:38--  https://github.com/aritter/CS-7650-sp22/raw/master/slides/aclImdb_small.tgz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aritter/CS-7650-sp22/master/slides/aclImdb_small.tgz [following]
--2022-01-10 23:36:39--  https://raw.githubusercontent.com/aritter/CS-7650-sp22/master/slides/aclImdb_small.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.

# Converting text to numbers

Below is some code we are providing you to read in the IMDB dataset, perform tokenization (using `nltk`), and convert words into indices.  Please don't modify this code in your submitted version.  We will provide example usage below.

In [ ]:
import os
import sys

import nltk
from nltk import word_tokenize
nltk.download('punkt')
import torch

#Sparse matrix implementation
from scipy.sparse import csr_matrix
import numpy as np
from collections import Counter

np.random.seed(1)

class Vocab:
    def __init__(self, vocabFile=None):
        self.locked = False
        self.nextId = 0
        self.word2id = {}
        self.id2word = {}
        if vocabFile:
            for line in open(vocabFile):
                line = line.rstrip('\n')
                (word, wid) = line.split('\t')
                self.word2id[word] = int(wid)
                self.id2word[wid] = word
                self.nextId = max(self.nextId, int(wid) + 1)

    def GetID(self, word):
        if not word in self.word2id:
            if self.locked:
                return -1        #UNK token is -1.
            else:
                self.word2id[word] = self.nextId
                self.id2word[self.word2id[word]] = word
                self.nextId += 1
        return self.word2id[word]

    def HasWord(self, word):
        return self.word2id.has_key(word)

    def HasId(self, wid):
        return self.id2word.has_key(wid)

    def GetWord(self, wid):
        return self.id2word[wid]

    def SaveVocab(self, vocabFile):
        fOut = open(vocabFile, 'w')
        for word in self.word2id.keys():
            fOut.write("%s\t%s\n" % (word, self.word2id[word]))

    def GetVocabSize(self):
        #return self.nextId-1
        return self.nextId

    def GetWords(self):
        return self.word2id.keys()

    def Lock(self):
        self.locked = True

class IMDBdata:
    def __init__(self, directory, vocab=None):
        """ Reads in data into sparse matrix format """
        pFiles = os.listdir("%s/pos" % directory)
        nFiles = os.listdir("%s/neg" % directory)

        if not vocab:
            self.vocab = Vocab()
        else:
            self.vocab = vocab

        #For csr_matrix (see http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)
        X_values = []
        X_row_indices = []
        X_col_indices = []
        Y = []

        XwordList = []
        XfileList = []

        #Read positive files
        for i in range(len(pFiles)):
            f = pFiles[i]
            for line in open("%s/pos/%s" % (directory, f)):
                wordList   = [self.vocab.GetID(w.lower()) for w in word_tokenize(line) if self.vocab.GetID(w.lower()) >= 0]
                XwordList.append(wordList)
                XfileList.append(f)
                wordCounts = Counter(wordList)
                for (wordId, count) in wordCounts.items():
                    if wordId >= 0:
                        X_row_indices.append(i)
                        X_col_indices.append(wordId)
                        X_values.append(count)
            Y.append(+1.0)

        #Read negative files
        for i in range(len(nFiles)):
            f = nFiles[i]
            for line in open("%s/neg/%s" % (directory, f)):
                wordList   = [self.vocab.GetID(w.lower()) for w in word_tokenize(line) if self.vocab.GetID(w.lower()) >= 0]
                XwordList.append(wordList)
                XfileList.append(f)
                wordCounts = Counter(wordList)
                for (wordId, count) in wordCounts.items():
                    if wordId >= 0:
                        X_row_indices.append(len(pFiles)+i)
                        X_col_indices.append(wordId)
                        X_values.append(count)
            Y.append(-1.0)
            
        self.vocab.Lock()

        #Create a sparse matrix in csr format
        self.X = csr_matrix((X_values, (X_row_indices, X_col_indices)), shape=(max(X_row_indices)+1, self.vocab.GetVocabSize()))
        self.Y = np.asarray(Y)

        #Randomly shuffle
        index = np.arange(self.X.shape[0])
        np.random.shuffle(index)
        self.X = self.X[index,:]
        self.XwordList = [torch.LongTensor(XwordList[i]) for i in index]  #Two different sparse formats, csr and lists of IDs (XwordList).
        self.XfileList = [XfileList[i] for i in index]
        self.Y = self.Y[index]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The code below reads the train,dev and test data into memory.  This will take a minute or so.

> Indented block



In [ ]:
train = IMDBdata("aclImdb_small/train")
train.vocab.Lock()
dev  = IMDBdata("aclImdb_small/dev", vocab=train.vocab)
test  = IMDBdata("aclImdb_small/test", vocab=train.vocab)

# Exploring the data

Below are a few examples to help you get familiar with how the data is represented.  $X \in \mathbb{R}^{N \times M}$ contains the design matrix and $Y \in \{+1,-1\}^N$ contains the labels.  Because there are a large number of features in this representation ($X$ contains one column representing each unique word in the dataset), we are using a sparse matrix representation (Numpy's [csr_sparse](http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)).  PyTorch doesn't have great support for sparse matrices...


In [ ]:
# X contains the design matrix representing the training data.
print(f"Train.X has {train.X.shape[0]} rows and {train.X.shape[1]} columns.")

Train.X has 7222 rows and 58162 columns.


In [ ]:
# Labels
train.Y

array([-1., -1., -1., ..., -1.,  1., -1.])

In [ ]:
# Let's count the frequency of every word appearing in the documents with negative sentiment:
word_counts = np.array(train.X[train.Y == -1,:].sum(axis=0)).flatten()
word_counts

array([    5, 20900,     1, ...,     1,     1,     1], dtype=int64)

In [ ]:
# Now, let's sort the words by frequency:
sorted_words = list(reversed(np.argsort(word_counts)))
sorted_words[0:10]

[17, 23, 16, 13, 1, 34, 74, 69, 66, 68]

In [ ]:
# What is the index of the most frequent word?
sorted_words[0]

17

In [ ]:
# Let's see what word that is:
train.vocab.GetWord(sorted_words[0])

'the'

In [ ]:
# What are the 10 most frequent words?
[train.vocab.GetWord(sorted_words[x]) for x in range(10)]

['the', ',', '.', 'a', 'and', 'to', 'of', '>', '<', '/']

# Now it's time to write some code!

# Basic Perceptron (10 points)

Implement the perceptron classification algorithm (fill in the missing code marked with `TODO:` below).
The only hyperparameter to tune is the number of iterations.
Accuracy > 80% is acceptable.

In [ ]:
class Eval:
    def __init__(self, pred, gold):
        self.pred = pred
        self.gold = gold
        
    def Accuracy(self):
        return np.sum(np.equal(self.pred, self.gold)) / float(len(self.gold))

class Perceptron:
    def __init__(self, X, Y, N_ITERATIONS):
        #TODO: Initalize parameters
        self.Train(X,Y)

    def ComputeAverageParameters(self):
        #TODO: Compute average parameters (do this part last)
        return

    def Train(self, X, Y):
        #TODO: Estimate perceptron parameters
        return

    def Predict(self, X):
        #TODO: Implement perceptron classification
        return [1 for i in range(X.shape[0])]

    def SavePredictions(self, data, outFile):
        Y_pred = self.Predict(data.X)
        fOut = open(outFile, 'w')
        for i in range(len(data.XfileList)):
          fOut.write(f"{data.XfileList[i]}\t{Y_pred[i]}\n")

    def Eval(self, X_test, Y_test):
        Y_pred = self.Predict(X_test)
        ev = Eval(Y_pred, Y_test)
        return ev.Accuracy()
    
ptron = Perceptron(train.X, train.Y, 10)
ptron.ComputeAverageParameters()
print(ptron.Eval(dev.X, dev.Y))

ptron.SavePredictions(test, 'test_pred_perceptron.txt')

print(train.X.shape)
print(test.X.shape)

#TODO: Print the 20 most positive and 20 most negative words

0.5
(7222, 58162)
(7222, 58162)


# Parameter Averaging (5 points)

Once your basic perceptron implementation is working, modify code to implement parameter averaging.  Instead of using parameters from the final iteration, $w_n$, to classify test examples, 
use the average of the parameters from every iteration, $\frac{1}{N}\sum_{i=1}^N w_i$.  A nice trick for doing this efficiently is described in section 2.1.1 of [Hal Daume's thesis](http://www.umiacs.umd.edu/~hal/docs/daume06thesis.pdf).  When you are done, save a copy of your predictions on the test set to turn in at the end (click on the file icon on the left side of the notebook).
For this part, accuracy > 82% is acceptable.

# Inspect Feature Weights (5 points)

Modify the code block above to print out the 20 most positive and 20 most negative words in the vocabulary sorted by their learned weights according to your model.  This will require a bit of thought how to do because the words in each document have been converted to IDs (see examples above).

# Evaluate on the test set

Once you are happy with your performance on the dev set, run the cell below to evaluate your performance on the test set.  Make sure to download the predictions of your model in `test_pred_perceptron.txt`.

In [ ]:
print(ptron.Eval(test.X, test.Y))

ptron.SavePredictions(test, 'test_pred_perceptron.txt')

0.5


# PyTorch Introduction

Take a look at the code below, which provides an example of how a simple neural network is capable of learning the XOR function (note that the perceptron cannot do this).  Refer to the PyTorch documentation [here](https://pytorch.org/docs/stable/index.html) for more information.

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import random
import numpy as np

#Define the computation graph; one layer hidden network
class FFNN(nn.Module):
    def __init__(self, dim_i, dim_h, dim_o):
        super(FFNN, self).__init__()
        self.V = nn.Linear(dim_i, dim_h)
        self.g = nn.Tanh()
        self.W = nn.Linear(dim_h, dim_o)
        self.logSoftmax = nn.LogSoftmax(dim=0)

    def forward(self, x):
        return self.logSoftmax(self.W(self.g(self.V(x))))

train_X = np.array([[0,0], [0,1], [1,0], [1,1]])
train_Y = np.array([0,1,1,0])

num_classes  = 2
num_hidden   = 10
num_features = 2

ffnn = FFNN(num_features, num_hidden, num_classes)
optimizer = optim.Adam(ffnn.parameters(), lr=0.1)

for epoch in range(100):
    total_loss = 0.0
    #Randomly shuffle examples in each epoch
    shuffled_i = list(range(0,len(train_Y)))
    random.shuffle(shuffled_i)
    for i in shuffled_i:
        x        = torch.from_numpy(train_X[i]).float()
        y_onehot = torch.zeros(num_classes)
        y_onehot[train_Y[i]] = 1

        ffnn.zero_grad()
        logProbs = ffnn.forward(x)
        loss = torch.neg(logProbs).dot(y_onehot)
        total_loss += loss
        
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:    
      print("loss on epoch %i: %f" % (epoch, total_loss))

#Evaluate on the training set:
num_errors = 0
for i in range(len(train_Y)):
    x = torch.from_numpy(train_X[i]).float()
    y = train_Y[i]
    logProbs = ffnn.forward(x)
    prediction = torch.argmax(logProbs)
    if y != prediction:
        num_errors += 1
print("number of errors: %d" % num_errors)

loss on epoch 0: 4.679323
loss on epoch 10: 2.623587
loss on epoch 20: 1.107152
loss on epoch 30: 0.215510
loss on epoch 40: 0.073905
loss on epoch 50: 0.041755
loss on epoch 60: 0.027791
loss on epoch 70: 0.020243
loss on epoch 80: 0.015303
loss on epoch 90: 0.012138
number of errors: 0


# Neural Bag-of-Words (10 points)

Your next task is to implement a neural bag-of-words baseline, NBOW-RAND, as described in [this paper](https://www.aclweb.org/anthology/P15-1162.pdf).  See section 2.1.  Note that the dataset we are using is a sample of the full IMDB dataset to make developing your solution easier, so your performance will be a bit lower than the numbers reported in the paper for the full dataset.

# GPUs

You probably want to use a GPU to enable faster training with larger word embeddings and hidden layers (the paper listed above uses 300 dimensions).

Colab provides free access to GPUs, which will be useful for rest of the assignment.  To access a GPU, select `Runtime` from the menu at the top of the page, and then choose `Change Runtime Type`; under the `Hardware Accelerator` dropdown select `GPU`.  Note that the free version of Colab does have quotas on how much GPU can be used - you won't need to use a GPU for the first part of the assignment (implementing the Perceptron algorithm).

You can check to make sure a GPU is available using the code below:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jan 23 17:05:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data format

In addition to reading in the data in sparse matrix (Numpy CSR) format, which you used in your implementation of the perceptron algorithm, the data loading code above also reads it in another format in `train.XwordList`.  This contains a list of PyTorch arrays consisting of sequences of word IDs corresponding to the content of each document.  The documents in `train.XwordList` are presented in the same order as labels (`train.Y`).  For the next part of the assignment, you should work with the data in this new format, which is fairly standard for text data in neural networks.  See below for a few examples on how to access the data in this new format.

In [ ]:
# Example of using XwordList

print("Here is the first document:")
print(train.XwordList[0].tolist())

print("After converting IDs to words:")
print([train.vocab.GetWord(x) for x in train.XwordList[0].tolist()])

print("Here is the label:")
print(train.Y[0])

Here is the first document:
[60, 128, 188, 17, 2737, 128, 174, 108, 332, 470, 87, 2051, 128, 16, 90, 174, 188, 3230, 385, 143, 17, 127, 128, 2133, 74, 615, 7335, 16, 99, 188, 779, 238, 17, 7567, 892, 3056, 115, 891, 1704, 74, 17, 17418, 1348, 34, 1645, 115, 1507, 23, 17, 2383, 23, 17, 4115, 23, 17, 9369, 23, 17, 3934, 16, 222, 566, 108, 332, 5230, 29749, 6079, 566, 549, 7567, 6836, 1194, 40780, 12, 17, 1348, 873, 30, 19705, 16, 115, 56, 891, 1868, 17, 7567, 892, 19705, 17, 17418, 1, 892, 1194, 12, 17, 1348, 18668, 17, 1598, 16, 174, 749, 17, 11858, 337, 423, 4910, 138, 5159, 12, 7567, 1, 106, 5308, 16, 174, 4112, 749, 174, 5991, 274, 1113, 12, 60, 32759, 174, 252, 200, 2274, 60, 128, 4878, 253, 667, 190, 99, 72, 219, 17, 7329, 526, 224, 10359, 99, 16]
After converting to IDs to words:
['this', 'movie', 'was', 'the', 'worst', 'movie', 'i', 'have', 'seen', 'since', '``', 'date', 'movie', '.', "''", 'i', 'was', 'laughing', 'through', 'out', 'the', 'whole', 'movie', 'instead', 'of', 'being

# Suggestions for getting started

We recommend following a similar structure as the XOR example above, starting with an [Embedding Layer](https://pytorch.org/docs/stable/nn.html\#embedding), a single hidden layer and [Adam optimizer.](https://pytorch.org/docs/stable/optim.html\#torch.optim.Adam)
Please refer to the Pytorch Documentation for more information as necessary.  You can use a batch size of one for this assignment, to simplify your implementation.
Feel free to change the number of iterations and learning rate. For this part, accuracy > 83% is acceptable.

In [ ]:
import tqdm

class NBOW(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=300, NUM_CLASSES=2):
        super(NBOW, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        #TODO: Initialize parameters.

    def forward(self, X):
        #TODO: Implement forward computation.
        return torch.randn(self.NUM_CLASSES)

def EvalNet(data, net):
    num_correct = 0
    Y = (data.Y + 1.0) / 2.0
    X = data.XwordList
    for i in range(len(X)):
        logProbs = net.forward(X[i])
        pred = torch.argmax(logProbs)
        if pred == Y[i]:
            num_correct += 1
    print("Accuracy: %s" % (float(num_correct) / float(len(X))))

def SavePredictions(data, outFile, net):
    fOut = open(outFile, 'w')
    for i in range(len(data.XwordList)):
        logProbs = net.forward(data.XwordList[i])
        pred = torch.argmax(logProbs)
        fOut.write(f"{data.XfileList[i]}\t{pred}\n")

def Train(net, X, Y, n_iter, dev):
    print("Start Training!")
    #TODO: initialize optimizer.

    num_classes = len(set(Y))

    for epoch in range(n_iter):
        num_correct = 0
        total_loss = 0.0
        net.train()   #Put the network into training mode
        for i in tqdm.notebook.tqdm(range(len(X))):
            pass
            #TODO: compute gradients, do parameter update, compute loss.
        net.eval()    #Switch to eval mode
        print(f"loss on epoch {epoch} = {total_loss}")
        EvalNet(dev, net)

nbow = NBOW(train.vocab.GetVocabSize()).cuda()
Train(nbow, train.XwordList, (train.Y + 1.0) / 2.0, 5, dev)

Start Training!



loss on epoch 0 = 0.0
Accuracy: 0.5117011701170117



loss on epoch 1 = 0.0
Accuracy: 0.5031503150315032



loss on epoch 2 = 0.0
Accuracy: 0.5076507650765076



loss on epoch 3 = 0.0
Accuracy: 0.4864986498649865



loss on epoch 4 = 0.0
Accuracy: 0.4990999099909991


# Evaluate on the test set

Once you are happy with your performance on the dev set, run the cell below to evaluate your performance on the test set.  Make sure to download the predictions of your model in `test_pred_nbow.txt`.

In [ ]:
EvalNet(test, nbow)
SavePredictions(test, 'test_pred_nbow.txt', nbow)

Accuracy: 0.5080310163389643


# 1-D Convolutional Neural Networks (2 points - optional extra credit)

The final task for this assignment is to implement a convolutional neural network for text classification similar to the CNN-rand baseline described by [Kim (2014)](https://www.aclweb.org/anthology/D14-1181.pdf).  We haven't covered CNNs in lecture yet, so this part is optional.  You should use the same data format as the NBOW model above, starting out with an [Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) layer, followed by a [1-D convolution](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html), a [max-pooling layer](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html#torch.nn.MaxPool1d) and a [Dropout layer](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html).  You should be able to use the same `Train()` function you wrote above. Accuracy > 83% is acceptable.

In [ ]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=300, NUM_CLASSES=2):
        super(CNN, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        #TODO: Initialize parameters.

    def forward(self, X):
        #TODO: Implement forward computation.
        return torch.randn(self.NUM_CLASSES)

cnn = CNN(train.vocab.GetVocabSize()).cuda()
Train(cnn, train.XwordList, (train.Y + 1.0) / 2.0, 5, dev)

Start Training!



loss on epoch 0 = 0.0
Accuracy: 0.495949594959496



loss on epoch 1 = 0.0
Accuracy: 0.4900990099009901



loss on epoch 2 = 0.0
Accuracy: 0.5027002700270027



loss on epoch 3 = 0.0
Accuracy: 0.495949594959496



loss on epoch 4 = 0.0
Accuracy: 0.5333033303330333


# Evaluate on the test set

Once you are happy with your performance on the dev set, run the cell below to evaluate your performance on the test set.  Make sure to download the predictions of your model in `test_pred_nbow.txt`.

In [ ]:
EvalNet(test, cnn)
SavePredictions(test, 'test_pred_cnn.txt', cnn)

Accuracy: 0.4933536416505123


## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_perceptron.txt
* test_pred_nbow.txt
* test_pred_cnn.txt (optional)
* TextClassification_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.

